In [1]:
import numpy as np
import pandas as pd

In [2]:
train=pd.read_csv(r"C:\Users\satwi\Downloads\Credit_Lead_Prediction-main\Credit_Lead_Prediction-main\project\train_cleaned.csv")
train.head()

,ID,Gender,Age,Region_Code,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead,Is_Entr
0,NNVBBKZB,0.0,4.290459,18,1,3.761200,0.0,13.860193,0.0,0.0,0
1,IDD62UNG,0.0,3.401197,27,0,3.465736,0.0,13.274205,0.0,0.0,0
2,HD3DSEMC,0.0,4.025352,18,1,3.258097,0.0,14.210464,1.0,0.0,0
3,BF3NC7KV,1.0,3.526361,20,0,2.944439,0.0,13.061453,0.0,0.0,0
4,TEASRWXV,0.0,3.401197,32,0,3.496508,0.0,13.695360,0.0,0.0,0


In [3]:
test=pd.read_csv(r"C:\Users\satwi\Downloads\Credit_Lead_Prediction-main\Credit_Lead_Prediction-main\project\test_cleaned.csv")
test.head()

,ID,Gender,Age,Region_Code,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Entr
0,VBENBARO,1.0,3.367296,4,0,3.218876,1.0,13.517598,0.0,0
1,CCMEWNKY,1.0,3.761200,18,1,3.891820,1.0,13.738129,0.0,0
2,VK3KGA9M,1.0,3.433987,20,0,2.639057,0.0,12.282798,0.0,0
3,TT8RPZVC,1.0,3.367296,22,0,3.496508,0.0,13.674028,0.0,0
4,SHQZEYTZ,0.0,3.367296,20,0,2.944439,0.0,13.395572,0.0,0


In [4]:
submission_format=pd.read_csv(r"C:\Users\satwi\Downloads\Credit_Lead_Prediction-main\Credit_Lead_Prediction-main\project\sample_submission_eyYijxG.csv")
submission_format.head()

,ID,Is_Lead
0,VBENBARO,0
1,CCMEWNKY,0
2,VK3KGA9M,0
3,TT8RPZVC,0
4,SHQZEYTZ,0


In [6]:
train=train.loc[train.drop('ID',axis=1).drop_duplicates().index]


In [7]:
##Splitting the training data
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
X=train.drop(['Is_Lead','ID'],axis=1)
y=train['Is_Lead']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=101)

In [11]:
##Using baseline LightGBM Classifier to assess viability
from lightgbm import LGBMClassifier
baseline_LGBM=LGBMClassifier(n_estimators=100)
baseline_LGBM.fit(X_train,y_train)
roc_auc_score(y_test,baseline_LGBM.predict(X_test))

[LightGBM] [Info] Number of positive: 46639, number of negative: 149912
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010558 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 430
[LightGBM] [Info] Number of data points in the train set: 196551, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.237287 -> initscore=-1.167611
[LightGBM] [Info] Start training from score -1.167611


0.7339933895375764

In [12]:
##Using baseline XGBClassifier to assess viability
from xgboost import XGBClassifier
baseline_XGB=XGBClassifier(n_estimators=300)
baseline_XGB.fit(X_train,y_train)
roc_auc_score(y_test,baseline_XGB.predict(X_test))

0.7241317464877081

In [15]:
##Using baseline CatBoost Classifier to assess viability
from catboost import CatBoostClassifier
baseline_CBC=CatBoostClassifier(n_estimators=500)
baseline_CBC.fit(X_train,y_train)
roc_auc_score(y_test,baseline_CBC.predict(X_test))

Learning rate set to 0.185478
0:	learn: 0.5759426	total: 185ms	remaining: 1m 32s
1:	learn: 0.5073266	total: 227ms	remaining: 56.5s
2:	learn: 0.4669287	total: 270ms	remaining: 44.8s
3:	learn: 0.4424124	total: 301ms	remaining: 37.3s
4:	learn: 0.4275469	total: 338ms	remaining: 33.4s
5:	learn: 0.4180197	total: 374ms	remaining: 30.8s
6:	learn: 0.4117288	total: 411ms	remaining: 29s
7:	learn: 0.4078189	total: 450ms	remaining: 27.7s
8:	learn: 0.4051654	total: 492ms	remaining: 26.8s
9:	learn: 0.4034594	total: 528ms	remaining: 25.9s
10:	learn: 0.4021383	total: 565ms	remaining: 25.1s
11:	learn: 0.4014008	total: 597ms	remaining: 24.3s
12:	learn: 0.4009861	total: 634ms	remaining: 23.8s
13:	learn: 0.4004476	total: 672ms	remaining: 23.3s
14:	learn: 0.4000911	total: 705ms	remaining: 22.8s
15:	learn: 0.3999081	total: 739ms	remaining: 22.3s
16:	learn: 0.3995731	total: 774ms	remaining: 22s
17:	learn: 0.3993077	total: 808ms	remaining: 21.6s
18:	learn: 0.3990994	total: 846ms	remaining: 21.4s
19:	learn: 0.3

0.7284706072718292

In [16]:
models=[('LGBM',LGBMClassifier(n_estimators=100)),
       ('XGB',XGBClassifier(n_estimators=300)),
       ('CBC',CatBoostClassifier(n_estimators=500))]

In [17]:
##All three models are viable
##Creating an ensemble
from sklearn.ensemble import StackingClassifier

In [18]:
ensemble=StackingClassifier(estimators=models)
ensemble.fit(X,y)

[LightGBM] [Info] Number of positive: 58287, number of negative: 187402
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010116 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 430
[LightGBM] [Info] Number of data points in the train set: 245689, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.237239 -> initscore=-1.167877
[LightGBM] [Info] Start training from score -1.167877
Learning rate set to 0.20402
0:	learn: 0.5660699	total: 41.2ms	remaining: 20.6s
1:	learn: 0.4957759	total: 68.6ms	remaining: 17.1s
2:	learn: 0.4562026	total: 103ms	remaining: 17.1s
3:	learn: 0.4342547	total: 145ms	remaining: 18s
4:	learn: 0.4212453	total: 187ms	remaining: 18.6s
5:	learn: 0.4132864	total: 229ms	remaining: 18.8s
6:	learn: 0.4082603	total: 260ms	remaining: 18.3s
7:	learn: 0.4051110	total: 294ms	remaining: 18.1s
8:	learn: 

StackingClassifier(estimators=[('LGBM', LGBMClassifier()),
                               ('XGB',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              gpu_id=None, grow_policy=None,
                                              importance_type=None,
                                              interaction_...
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=300, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=None, ...)),
                               ('CBC',
                                <catboost.core.CatBoostClassifier object at 0x00000289CFF858D0>)])

In [19]:
submission_format.to_csv('final_submission.csv',index=False)